In [ ]:
import wave
import os
import librosa

In [ ]:
Path_to_train = "train\wav"
subfolders = os.listdir(Path_to_train)
data = []
for s in subfolders:
    files = os.listdir(Path_to_train + "/" +s)
    data.extend([Path_to_train + "/" + s+ "/" + f for f in files])
data[:10]

In [ ]:
#read text from every transcription audio
def read_text( text_path):
    text = []
    with open(text_path) as fp:
        line = fp.readline()
        while line:
        # TODO: fix spaces in in amharic text
            text.append(line)
            line = fp.readline()
    return text

In [ ]:
#extract the transcription and the label 
label=[]
transcriptions = []
for t in text:
    sp = t.split("\t")
    sp = sp.strip("\n")
    if len(sp) > 1:
        label.append(sp[0])
        transcriptions.append(sp[1])
transcriptions[:10]

In [ ]:
#get audio path , every path must corespond to transcription , get the transprion in the doc and append to audio path 
audio_path=[0]*len(transcriptions)
for d in data:
    _d = d.strip(".wav")
    sp = _d.split("/")[2]
    index = label.index(sp)
    audio_path[index] = d

In [ ]:
#calculate duration 
duration_of_recordings=[]
for d in audio_path:
    audio, fs = librosa.load(d, sr=None)
    duration_of_recordings.append(float(len(audio)/fs))

In [ ]:
import pandas as pd 
data=pd.DataFrame({'key': audio_path,'text': transcriptions, 'duration':duration_of_recordings})

# Resize and standardize
We resize all the audio samples to have the same length by either extending its duration by padding it with silence, or by truncating it.
We use the right padding method

Standardize the sample rate.


In [ ]:
# used to load audio file
#specifying sample rate will resize all the files i.e Audio will be automatically resampled to the given rate
class Loader:
  def __init__(self, sample_rate,duration,mono):
    self.sample_rate=sample_rate
    self.duration=duration
    self.mono=mono
    self.channel = 2

  def load_signal(self, filepath):
    signal=librosa.load(filepath,
                        sr=None,
                        duration=self.duration,
                        mono=self.mono)[0]    #librosa returns 2D array (signal,sample_rate) pick the signal here
    return signal

  def load_sample_rate(self,filepath):
    self.sample_rate=librosa.load(filepath,
                        sr =None,
                        duration=self.duration,
                        mono=self.mono)[1]    #librosa returns 2D array (signal,sample_rate) pick the sample rate here
    return self.sample_rate

  # def load(self,filepath):     #get both signal and sample rate in one
  #   aud=librosa.load(filepath,
  #                       sr=None,
  #                       duration=self.duration,
  #                       mono=self.mono)
  #   return aud

  def load(self,filepath):
    sig, sr = torchaudio.load(filepath)
    aud = sig, sr
    return aud

  #before using this function kindly change your file paths for it to work


  def make_stereo(self,aud):
      #this function converts mono audio channels into stereo channels 
  #     logging.info(" ============ Conerting audio sample from mono to stereo ================= ")
      print("======= Mono to stereo audio conversion")
      ifile = aud
      #log the info on adio files
  #     logging.info(ifile.getparams())
      print (ifile.getparams())
      # (1, 2, 44100, 2013900, 'NONE', 'not compressed')
      (nchannels, sampwidth, framerate, nframes, comptype, compname) = ifile.getparams()
      assert (comptype == 'NONE')  # Compressed not supported yet
      array_type = {1:'B', 2: 'h', 4: 'l'}[sampwidth]
      print(" ======= Calculting left channel type =====")
      left_channel = array.array(array_type, ifile.readframes(nframes))[::nchannels]
      ifile.close()

      #convert the number of channels to 2
      print("====== converting channels ======= ")
      stereo = 2 * left_channel
      stereo[0::2] = stereo[1::2] = left_channel
      #overwrite the wav file making it a stereo file
      print("====== overwriting wav file ======= ")
      ofile = wave.open(audio_path, 'w')
      ofile.setparams((2, sampwidth, framerate, nframes, comptype, compname))
      ofile.writeframes(stereo.tostring())
      ofile.close()

  def resample(self,aud):                    #standardize sample rate
    sig, sr = aud
    
    if (sr == self.sample_rate):
      # Nothing to do
      return aud[0]

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, self.sample_rate)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, self.sample_rate)(sig[1:,:])
      resig = torch.cat([resig, retwo])
      aud = resig, self.sample_rate
    return aud

In [ ]:
#resizes audios to have same length
class Padder:
  def __init__(self,mode="constant"):
    self.mode=mode
  def left_pad(self,array,num_missing_items):
    padded_array=np.pad(array,
                        (num_missing_items, 0),
                        mode=self.mode)
    return padded_array
  def right_pad(self,array,num_missing_items):
    padded_array=np.pad(array,
                        (0,num_missing_items),
                        mode=self.mode)
    return padded_array
  

In [ ]:
class PreprocessingPipeline:
  '''Processes audio files in a directory by applying the following steps
    1. Load the data, convert to stereo and resample sampling rate
    2. Pad the audio
  '''
  def __init__(self):
    self.padder=None
    self._loader=None
   

  def process(self,audio_files_directory):
    for root, directories, files in os.walk(audio_files_directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            self._process_file(filepath)
            print(f"Processed file {filepath}")
    
  def _process_file(self,filepath):
    signal=self.loader.load(filepath)
    signal = self.loader.make_stereo(signal)
    signal = self.loader.resample(signal)[0]
    if self._is_padding_necessary(signal):
      signal=self._apply_padding(signal)
    

  def _is_padding_necessary(self,signal):
    self.num_expected_samples=int(loader.sample_rate*loader.duration)
    if len(signal) < self.num_expected_samples:
      return True
    return False

  def _apply_padding(self,signal):
    num_missing_samples=self.num_expected_samples - len(signal)
    padded_signal = self.padder.right_pad(signal, num_missing_samples)
    return padded_signal


In [ ]:
DURATION=0.74
SAMPLE_RATE=22050
MONO=True

In [ ]:
loader=Loader(SAMPLE_RATE, DURATION, MONO)
padder=Padder()

In [ ]:
preprocessing_pipeline=PreprocessingPipeline()
preprocessing_pipeline.loader=loader
preprocessing_pipeline.padder=padder

In [ ]:
preprocessing_pipeline.process(Path_to_train)